In [1]:
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr10.fa.gz


--2025-10-27 19:07:09--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr10.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43157332 (41M) [application/x-gzip]
Saving to: 'chr10.fa.gz'

chr10.fa.gz         100%[===================>]  41.16M   945KB/s    in 2m 2s   

2025-10-27 19:09:18 (346 KB/s) - 'chr10.fa.gz' saved [43157332/43157332]



In [2]:
!wget [URL_TO_ILLUMINA_FILE] -O illumina.fq.bz2



zsh:1: no matches found: [URL_TO_ILLUMINA_FILE]


In [6]:
!bunzip2 -k illumina.fq.bz2



In [7]:
!bunzip2 -k pacbio.fq.bz2


In [8]:
!minimap2 -ax sr chr10.fa illumina.fq | samtools sort -o illumina.bam


[M::mm_idx_gen::2.313*0.94] collected minimizers
[M::mm_idx_gen::2.680*1.21] sorted minimizers
[M::main::2.680*1.21] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::2.680*1.21] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 1
[M::mm_idx_stat::2.809*1.20] distinct minimizers: 19023001 (96.65% are singletons); average occurrences: 1.171; average spacing: 6.004; total length: 133797422
[M::worker_pipeline::10.715*2.40] mapped 309505 sequences
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -ax sr chr10.fa illumina.fq
[M::main] Real time: 10.759 sec; CPU: 25.739 sec; Peak RSS: 0.882 GB


In [9]:
!samtools index illumina.bam


In [10]:
!minimap2 -ax map-hifi chr10.fa pacbio.fq | samtools sort -o pacbio.bam


[M::mm_idx_gen::1.967*0.94] collected minimizers
[M::mm_idx_gen::2.247*1.18] sorted minimizers
[M::main::2.247*1.18] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::2.418*1.15] mid_occ = 110
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1
[M::mm_idx_stat::2.477*1.14] distinct minimizers: 11183865 (96.18% are singletons); average occurrences: 1.195; average spacing: 10.008; total length: 133797422
[M::worker_pipeline::3.540*1.64] mapped 3063 sequences
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -ax map-hifi chr10.fa pacbio.fq
[M::main] Real time: 3.572 sec; CPU: 5.823 sec; Peak RSS: 0.755 GB


In [11]:
!samtools index pacbio.bam


In [12]:
ls -lh *.bam *.bai


-rw-r--r--@ 1 tomislavstojcic  staff    20M Oct 27 19:40 illumina.bam
-rw-r--r--@ 1 tomislavstojcic  staff    79K Oct 27 19:40 illumina.bam.bai
-rw-r--r--@ 1 tomislavstojcic  staff   2.7M Oct 27 19:40 pacbio.bam
-rw-r--r--@ 1 tomislavstojcic  staff    58K Oct 27 19:40 pacbio.bam.bai


In [13]:
!bcftools mpileup -f chr10.fa illumina.bam | bcftools call -mv -Oz -o illumina.vcf.gz


Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250


In [14]:
!bcftools index illumina.vcf.gz


In [15]:
!bcftools mpileup -f chr10.fa pacbio.bam | bcftools call -mv -Oz -o pacbio.vcf.gz


Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250


In [16]:
!bcftools index pacbio.vcf.gz


In [18]:
!whatshap phase -o illumina_phased.vcf --reference chr10.fa illumina.vcf.gz illumina.bam


zsh:1: command not found: whatshap


In [19]:
!whatshap phase -o illumina_phased.vcf --reference chr10.fa illumina.vcf.gz illumina.bam


This is WhatsHap 2.8 running under Python 3.11.5
Working on 1 sample from 1 family

# Working on contig chr10 in individual illumina.bam
Found 1301 usable heterozygous variants (0 skipped due to missing genotypes)
Found 0 reads covering 0 variants
Kept 0 reads that cover at least two variants each
Selected 0 most phase-informative reads covering 0 variants
Best-case phasing would result in 0 non-singleton phased blocks (0 singletons). 
Phasing 1 sample by solving the MEC problem ...
No. of phased blocks: 0

# Resource usage
Time spent reading BAM/CRAM:                    0.3 s
Time spent parsing VCF:                         0.0 s
Time spent selecting reads:                     0.0 s
Time spent phasing:                             0.0 s
Time spent writing VCF:                         0.0 s
Time spent finding components:                  0.0 s
Time spent on rest:                             0.0 s
Total elapsed time:                             0.3 s


In [20]:
!whatshap phase -o pacbio_phased.vcf --reference chr10.fa pacbio.vcf.gz pacbio.bam


This is WhatsHap 2.8 running under Python 3.11.5
Working on 1 sample from 1 family

# Working on contig chr10 in individual pacbio.bam
Found 352 usable heterozygous variants (0 skipped due to missing genotypes)
Found 0 reads covering 0 variants
Kept 0 reads that cover at least two variants each
Selected 0 most phase-informative reads covering 0 variants
Best-case phasing would result in 0 non-singleton phased blocks (0 singletons). 
Phasing 1 sample by solving the MEC problem ...
No. of phased blocks: 0

# Resource usage
Time spent reading BAM/CRAM:                    0.3 s
Time spent parsing VCF:                         0.0 s
Time spent selecting reads:                     0.0 s
Time spent phasing:                             0.0 s
Time spent writing VCF:                         0.0 s
Time spent finding components:                  0.0 s
Time spent on rest:                             0.0 s
Total elapsed time:                             0.3 s


In [21]:
!bgzip illumina_phased.vcf


In [22]:
!bcftools index illumina_phased.vcf.gz


In [23]:
!bgzip pacbio_phased.vcf


In [24]:
!bcftools index pacbio_phased.vcf.gz


In [25]:
!bcftools isec -p isec_output illumina_phased.vcf.gz pacbio_phased.vcf.gz


In [26]:
!echo "Illumina-only variants:"


Illumina-only variants:


In [27]:
!wc -l isec_output/0000.vcf


    1437 isec_output/0000.vcf


In [28]:
!echo "PacBio-only variants:"


PacBio-only variants:


In [29]:
!wc -l isec_output/0001.vcf


     264 isec_output/0001.vcf


In [30]:
!echo "Shared variants:"


Shared variants:


In [31]:
!wc -l isec_output/0002.vcf


     569 isec_output/0002.vcf


In [1]:
print("CYP2C19 variant count:")
!bcftools view -H -r chr10:94759681-94853205 illumina_cyp_phased.vcf.gz | wc -l


CYP2C19 variant count:
     125


In [2]:
print("\nCYP2C9 variant count:")
!bcftools view -H -r chr10:94938683-94993415 illumina_cyp_phased.vcf.gz | wc -l



CYP2C9 variant count:
      71


In [3]:
print("\nCYP2C8 variant count:")
!bcftools view -H -r chr10:94842701-94924000 illumina_cyp_phased.vcf.gz | wc -l



CYP2C8 variant count:
     129


## IGV Analysis of Discordant Variants

### Variant 1: CYP2C19 chr10:94,772,788 
![IGV Screenshot 1](Screenshot%202025-11-02%20at%207.34.32%20PM.png)

**Technology:** Illumina only  
**Variant:**  
**Analysis:** This variant shows strong Illumina support with good coverage. PacBio has no reads in this region, suggesting a coverage gap rather than a sequencing artifact. This is likely a real variant.

---

### Variant 2: CYP2C19 chr10:94,770,084
![IGV Screenshot 2](Screenshot%202025-11-02%20at%207.38.35%20PM.png)

**Coverage:** Illumina: 35X, PacBio: 34X  
**Analysis:** Illumina shows a large indel (blue/green colored block) with strong support across multiple reads. PacBio has adequate coverage but shows only minimal variant support. The PacBio reads appear to disagree on the exact nature of this variant. Most likely a real indel that Illumina captures clearly but PacBio has difficulty resolving, possibly due to alignment challenges in this region.

---

### Variant 3: CYP2C9 chr10:94,969,284 
![IGV Screenshot 3](Screenshot%202025-11-02%20at%207.39.33%20PM.png)
**Coverage:** Illumina: 51X, PacBio: 37X  
**Analysis:** PacBio shows 3 blue C's stacked at this position, representing only 3 out of 37 reads supporting this variant (8% allele frequency). Illumina data with 51X coverage shows no variant at this position. Thus this is probably not a true variant.

---

### Variant 4: CYP2C9 chr10:94,969,284 
![IGV Screenshot 4](Screenshot%202025-11-02%20at%207.39.49%20PM.png)

**Coverage:** Illumina: 43X, PacBio: 1X  
**Analysis:** Illumina shows a large indel with strong support across 43 reads. PacBio has only 1 read in this region. The indel appears real based on consistent Illumina support, but cannot be validated due to insufficient PacBio coverage. Thus Likely a true variant; discordance due to PacBio coverage gap, not sequencing error.

---

### Variant 5: CYP2C8 chr10:94,847,100 
![IGV Screenshot 5](Screenshot%202025-11-02%20at%207.40.17%20PM.png)

**Coverage:** Illumina: 55X, PacBio: 26X  
**Analysis:** Illumina shows multiple SNVs with strong read support across the region. PacBio has good coverage but shows minimal variant calls. Illumina variants likely true; PacBio single-read variant likely **artifact**. Discordance due to **variant calling sensitivity differences** between technologies, not sequencing errors.

In [9]:
print("=== CYP2C19 variants ===")
!bcftools query -f '%POS\t%REF\t%ALT\t[%GT]\n' -r chr10:94759681-94853205 illumina_cyp_phased.vcf.gz

=== CYP2C19 variants ===
94760573	C	A	0/1
94761900	C	T	0/1
94762804	C	T	1/1
94763232	A	T	0/1
94765362	G	A	0/1
94765779	G	A	1/1
94766304	ttg	tTGTAtg	1/1
94766352	G	A	0/1
94767381	A	C	1/1
94767577	C	A	0/1
94767965	C	G	1/1
94768394	G	A	0/1
94770155	A	G	1/1
94772788	G	T	1/1
94772850	T	C	1|0
94772907	G	A	1|0
94772931	G	A	0|1
94773293	A	T	1/1
94774827	G	A	1/1
94775961	G	A	0/1
94776698	T	A	0|1
94776951	C	T	0|1
94777632	A	G	0/1
94778957	T	A	1|0
94778981	G	A	0|1
94779551	tttttcttttcttttctttt	tTTTTCTTTTCttttcttttcttttctttt	0|1
94782837	C	T	1|0
94783106	A	G	1|0
94784235	cttttt	cTttttt	1/1
94784708	G	A	1|0
94784970	C	G	1|0
94785981	C	T	0/1
94787917	G	A	0/1
94788106	gttccattcca	gttcca	0|1
94788559	T	C	0|1
94789209	A	G	0|1
94791495	G	T	0/1
94792552	G	T	0/1
94793025	A	C	1/1
94794531	A	T	1/1
94794659	A	G	1/1
94795304	T	G	0|1
94795486	C	A	0|1
94795832	C	G	0|1
94797351	T	C	0/1
94798522	C	G	1|0
94798814	atttttttttttttttttttttttt	atttttttttttttttttttttt	0|1
94801232	C	T	0|1
94801768	A	G	0|1
94801966	C	A	0

In [10]:
print("\n=== CYP2C9 variants ===")
!bcftools query -f '%POS\t%REF\t%ALT\t[%GT]\n' -r chr10:94938683-94993415 illumina_cyp_phased.vcf.gz



=== CYP2C9 variants ===
94938933	T	C	0/1
94939926	G	A	0/1
94940780	C	T	1/1
94940873	A	T	1/1
94941267	gtttttttttttttttttttttttttt	gTTtttttttttttttttttttttttttt	1/1
94941320	G	T	0/1
94942093	T	C	0/1
94943734	C	A	0/1
94944819	A	G	1/1
94945466	A	T	0/1
94946130	T	C	1|0
94946131	G	T	1|0
94946177	A	G	1/1
94946652	C	G	0/1
94950236	T	C	1/1
94950968	C	T	1/1
94952127	G	A	0|1
94952386	T	C	1|0
94952643	A	G	1/1
94952785	tatagata	tATAGatagata	0|1
94953443	T	C	0/1
94954003	G	A	0/1
94955914	T	C	1|0
94956377	C	T	1|0
94956617	A	C	1|0
94957439	T	G	1/1
94958456	T	C	1|0
94959037	C	T	1|0
94959052	G	T	0|1
94959431	A	G	1/1
94959719	A	G	1/1
94960070	A	G	1/1
94960275	T	C	1/1
94960761	A	G	0/1
94961350	G	C	1/1
94962302	C	A	1/1
94963667	A	G	0/1
94964413	T	C	1/1
94966538	tcccc	tccc	0/1
94967800	G	A	1|0
94968273	C	T	1|0
94968463	A	C	1|0
94968568	T	G	1|0
94968623	gttttttttttt	gtttttttttt	0|1
94969284	A	C	0/1
94970667	G	A	1|0
94971060	T	A	0|1
94971535	C	T	1|0
94972340	A	G	0/1
94972974	T	G	1/1
94973780	T	C	1/1
94974535

In [11]:
print("\n=== CYP2C8 variants ===")
!bcftools query -f '%POS\t%REF\t%ALT\t[%GT]\n' -r chr10:94842701-94924000 illumina_cyp_phased.vcf.gz


=== CYP2C8 variants ===
94842866	A	G	1/1
94843657	A	G	0|1
94844516	C	G	0|1
94845570	G	C	1/1
94848458	C	T	0/1
94849307	G	A	1/1
94850760	T	C	1/1
94851443	Caaataaataaataaataaataaataaataaataaataaataaataaataa	Caaataaataaataaataaataaataaataaataaataaataaataa	0|1
94851871	T	C	0|1
94852077	G	A	1/1
94852283	G	A	0|1
94852614	C	T	0|1
94852716	A	G	1|0
94853547	Ctttttttttttttttt	CTtttttttttttttttt	1/1
94854102	tccacc	tcc	0/1
94854968	G	T	0/1
94856167	C	T	0/1
94856660	A	T	1/1
94857274	T	G	0/1
94858114	A	G	0/1
94859474	C	T	0/1
94859761	GCCCCCCCC	GCCCCCCCCCC,GCCCCCCCCC	1/2
94859849	G	A	1/1
94860797	A	G	1/1
94860863	caaaaaaaaa	cAaaaaaaaaa	0/1
94861449	G	C	1/1
94861942	T	C	0/1
94864151	gtttttttttttt	gTtttttttttttt	0/1
94866231	C	T	1|0
94866445	T	C	0|1
94866498	G	A	0|1
94868517	T	C	0|1
94868852	C	A	0|1
94869480	A	C	1|0
94870493	caaaaaaaaaaaaaa	caaaaaaaaaaaaa	0|1
94870503	A	T	1|0
94872496	T	C	1/1
94873074	A	G	0/1
94874084	C	T	0/1
94876348	T	C	0/1
94877910	A	G	1/1
94878357	C	T	0/1
94879614	T	A	1/1
94880345

### Star-Allele Calls

**CYP2C19: likely *1/*1** because:
- Did not find the major star-allele defining variants at their expected positions
- CYP2C19*2 (rs4244285): not found
- CYP2C19*17 (rs12248560): not found


**CYP2C9: likely *1/*1** because:
- Did NOT find the major star-allele defining variants
- CYP2C9*2 (rs1799853): not found  
- CYP2C9*3 (rs1057910): not found

**CYP2C8: uncertain** because:
- FOUND variant at position 94842866 (A>G, homozygous) matching rs11572080 (R139K), a CYP2C8*3 defining variant
- Did NOT find the second *3 variant (rs10509681, K399R) at expected position


# AI Usage Documentation


## Summary of Usage
Claude Sonnet 4.5 was used throughout this assignment for code generation and result interpretation. 

---

## Detailed Usage by Section

### 1. Reference Genome Download
  - Commands for downloading chr10 from UCSC/Ensembl
  - wget/curl syntax
  - FASTA file handling

### 2. Alignment (minimap2)
  - minimap2 parameter selection for Illumina vs PacBio data
  - Proper preset flags (-ax sr vs -ax map-pb)
  - SAM to BAM conversion and indexing commands
  

### 3. Variant Calling (bcftools)
  - bcftools mpileup and call command syntax
  - Parameter selection 
  - VCF filtering and normalization
  

### 4. Phasing (HapCUT2/WhatsHap)
  - Installation instructions for phasing tools
  - Command syntax for HapCUT2 and WhatsHap
  - Converting HapCUT2 block format to phased VCF
  - Troubleshooting phasing errors

### 5. Variant Analysis & Comparison
  - bcftools isec commands for VCF comparison
  - Extracting shared vs unique variants
  - IGV automation scripting
  - Interpreting IGV screenshots


### 6. Star-Allele Determination
  - Understanding PharmVar database structure
 
